<a href="https://colab.research.google.com/github/marco-siino/ThingSpeak_ParsersGenerator/blob/main/Gemma_7B_Instruct_ThingSpeak_ParsersGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
!nvidia-smi

Thu Jun 20 01:27:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   43C    P0               74W / 461W|  10423MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Generate the Python code to perform the task.

In [3]:
prompt ="""
Write the Python code to convert the following JSON FILE using the following JSON SCHEMA:

JSON FILE:

{
    "channel": {
        "id": 1293177,
        "name": "San Diego - Estaci\u00f3n Meteorol\u00f3gica",
        "description": "San Diego, Cerro Largo, Uruguay\r\nEstaci\u00f3n Meteorol\u00f3gica Solar\r\n(Temp, Hum, Presion, Lluvia, Viento).\r\nESP8266, UNO R3, BME 680\r\nUpdate Interval - 15 seg\r\nhttps://clima.santiago.ovh/",
        "latitude": "-31.9939484",
        "longitude": "-53.9575388",
        "field1": "Temperatura C\u00b0",
        "field2": "Humedad %",
        "field3": "Pres. Atmosf\u00e9rica (hPa)",
        "field4": "Gas",
        "field5": "Viento",
        "field6": "Precipitaci\u00f3n (mm)",
        "field7": "Direcci\u00f3n del Viento",
        "field8": "UV",
        "created_at": "2021-01-30T16:32:32Z",
        "updated_at": "2024-06-18T14:05:34Z",
        "elevation": "136",
        "last_entry_id": 4502987
    },
    "feeds": [
        {
            "created_at": "2024-06-18T13:33:39Z",
            "entry_id": 4502888,
            "field1": "17.73",
            "field2": "93.32",
            "field3": "996.29",
            "field4": "103.33",
            "field5": "0.00",
            "field6": "0",
            "field7": "0",
            "field8": "0.74"
        }
    ]
}

JSON SCHEMA:

{
    "type": "object",
    "properties": {
      "sensorId": {
        "type": "string"
      },
      "timestamp": {
        "type": "string",
        "format": "date-time"
      },
      "temperature": {
        "type": "number"
      },
      "unit": {
        "type": "string",
        "enum": ["Celsius", "Fahrenheit"]
      }
    },
    "required": ["sensorId", "timestamp", "temperature", "unit"]
  }



"""

chat = [
    { "role": "user", "content": prompt }
]


prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1500)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Write the Python code to convert the following JSON FILE using the following JSON SCHEMA:

JSON FILE:

{
    "channel": {
        "id": 1293177,
        "name": "San Diego - Estación Meteorológica",
        "description": "San Diego, Cerro Largo, Uruguay
Estación Meteorológica Solar
(Temp, Hum, Presion, Lluvia, Viento).
ESP8266, UNO R3, BME 680
Update Interval - 15 seg
https://clima.santiago.ovh/",
        "latitude": "-31.9939484",
        "longitude": "-53.9575388",
        "field1": "Temperatura C°",
        "field2": "Humedad %",
        "field3": "Pres. Atmosférica (hPa)",
        "field4": "Gas",
        "field5": "Viento",
        "field6": "Precipitación (mm)",
        "field7": "Dirección del Viento",
        "field8": "UV",
        "created_at": "2021-01-30T16:32:32Z",
        "updated_at": "2024-06-18T14:05:34Z",
        "elevation": "136",
        "last_entry_id": 4502987
    },
    "feeds": [
        {
            "created_at": "2024-06-18T13:33:39

# Let's run the code.

In [6]:
import json

# Read the JSON file
with open("data.json") as f:
    data = json.load(f)

# Convert the JSON data to the specified schema
schema = {
    "type": "object",
    "properties": {
        "sensorId": {"type": "string"},
        "timestamp": {"type": "string", "format": "date-time"},
        "temperature": {"type": "number"},
        "unit": {"type": "string", "enum": ["Celsius", "Fahrenheit"]}
    },
    "required": ["sensorId", "timestamp", "temperature", "unit"]
}

converted_data = json.dumps(data, schema)

# Print the converted data
print(converted_data)

TypeError: dumps() takes 1 positional argument but 2 were given

In [ ]:
for filename in test_set:
        file_path = folder_name+filename
        with open(os.path.join(folder_path, filename), 'r') as file:
          file_contents = file.read()
          find_duplicate_frames(log_data)
        break

ValueError: not enough values to unpack (expected 2, got 0)

## Let's run the code.

In [ ]:
!rm -r 'mistral_7b_source_code_results/'

In [ ]:
# Define the directory path containing the results.
folder_name = 'mistral_7b_Q2_source_code_results/'
#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = folder_name+filename
        print(filename)
        print(file_path)
        if(os.path.isfile(file_path)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)

                result = process_log(file_contents[0])

                print(result+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=268.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=268.txt
For DEVICE dev_eui:120 all received frames are 4.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=924.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=924.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 1.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 1.
For DEVICE dev_eui:125 all received fra

# Here the function to run the evaluation at file-level and at global-level.

In [ ]:
def evaluate_results (dataset_path,result_path):
  dataset_corpus = ""
  results_corpus = ""
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      dataset_corpus += text2
      results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  reference = results_corpus.split()
  candidate = dataset_corpus.split()

  # Compute BLEU score
  smoothing_function = SmoothingFunction().method1
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

  # Compute ROUGE score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  rouge_scores = scorer.score(text1, text2)

  # Print the results
  print(f"BLEU score: {bleu_score}")
  print(f"ROUGE-1: {rouge_scores['rouge1']}")
  print(f"ROUGE-L: {rouge_scores['rougeL']}")

## Let's evaluate the results of Mistral.

In [ ]:
dataset_path = "prompt_Q2/"
result_path = "mistral_7b_Q2_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
Sample:  prompt_n_hours=2_skip_hours=318.txt
dev_eui: 124 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)
ROUGE-L: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)



Sample:  prompt_n_hours=2_skip_hours=390.txt
dev_eui: 124 
 dev_eui: 126 
 dev_eui: 128


dev_eui: 123, dev_eui: 128, dev_eui: 124, dev_eui: 126
BLEU score: 0.07356556000975763
ROUGE-1: Score(precision=1.0, recall=0.75, fmeasure=0.8571428571428571)
ROUGE-L: Score(precision=0.6666666666666666, recall=0.5, fmeasure=0.5714285714285715)



Sample:  prompt_n_hours=2_skip_hours=710.txt
dev_eui: 120 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=0.8333333333333334, recall=0.5555555555555556, fmeasure=0.6666666666666667)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.555555555555555

## Let's evaluate the results of the source code generated by Mistral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "mistral_7b_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.
BLEU score: 1.0
ROUGE-1: Score(precision=1.0, recall=1.0, fmeasure=1.0)
ROUGE-L: Score(precision=1.0, recall=1.0, fmeasure=1.0)



Sample:  prompt_n_hours=2_skip_hours=362.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 